# 0. What is MsPacman

Maximize our score in the Atari 2600 game MsPacman.

In this environment, the observation is an RGB image of the screen, which is an array of shape (210, 160, 3) 
Each action is repeatedly performed for a duration of k frames, where k is uniformly sampled from {2,3,4}.

### Action Space
- The actual driving force is calculated by multiplying the power coef by power (0.0015)

 
### Observation Space
The observation space is a 2-dim vector:
- the 1st element represents the "car position" 
- the 2nd element represents the "car velocity"
 
### Rewards
- Reward of 100 is awarded if the agent reached the flag (position = 0.45) on top of the mountain. 
- Reward is decrease based on amount of energy consumed each step.

# 1. Import Dependencies

In [1]:
import gym
import time
from stable_baselines3 import A2C
from stable_baselines3.common.vec_env import VecFrameStack
from stable_baselines3.common.evaluation import evaluate_policy
from stable_baselines3.common.env_util import make_atari_env
import os

In [2]:
environment_name = 'MsPacman-v0'     # Using MsPacman verion 0 in this notebook

env = gym.make(environment_name)     # import the MsPacman from gym as environment

In [3]:
env.unwrapped.get_action_meanings()  # 9 actions could be taken in this environment

['NOOP',
 'UP',
 'RIGHT',
 'LEFT',
 'DOWN',
 'UPRIGHT',
 'UPLEFT',
 'DOWNRIGHT',
 'DOWNLEFT']

In [4]:
env.reset()                          # reset the evironment

array([[[  0,   0,   0],
        [  0,   0,   0],
        [  0,   0,   0],
        ...,
        [  0,   0,   0],
        [  0,   0,   0],
        [  0,   0,   0]],

       [[228, 111, 111],
        [228, 111, 111],
        [228, 111, 111],
        ...,
        [228, 111, 111],
        [228, 111, 111],
        [228, 111, 111]],

       [[228, 111, 111],
        [228, 111, 111],
        [228, 111, 111],
        ...,
        [228, 111, 111],
        [228, 111, 111],
        [228, 111, 111]],

       ...,

       [[  0,   0,   0],
        [  0,   0,   0],
        [  0,   0,   0],
        ...,
        [  0,   0,   0],
        [  0,   0,   0],
        [  0,   0,   0]],

       [[  0,   0,   0],
        [  0,   0,   0],
        [  0,   0,   0],
        ...,
        [  0,   0,   0],
        [  0,   0,   0],
        [  0,   0,   0]],

       [[  0,   0,   0],
        [  0,   0,   0],
        [  0,   0,   0],
        ...,
        [  0,   0,   0],
        [  0,   0,   0],
        [  0,   0,   0]]

In [5]:
env.action_space                

Discrete(9)

In [6]:
env.observation_space        # observation space in this environment 210x160 pixel, 3 RGB colors channel

Box([[[0 0 0]
  [0 0 0]
  [0 0 0]
  ...
  [0 0 0]
  [0 0 0]
  [0 0 0]]

 [[0 0 0]
  [0 0 0]
  [0 0 0]
  ...
  [0 0 0]
  [0 0 0]
  [0 0 0]]

 [[0 0 0]
  [0 0 0]
  [0 0 0]
  ...
  [0 0 0]
  [0 0 0]
  [0 0 0]]

 ...

 [[0 0 0]
  [0 0 0]
  [0 0 0]
  ...
  [0 0 0]
  [0 0 0]
  [0 0 0]]

 [[0 0 0]
  [0 0 0]
  [0 0 0]
  ...
  [0 0 0]
  [0 0 0]
  [0 0 0]]

 [[0 0 0]
  [0 0 0]
  [0 0 0]
  ...
  [0 0 0]
  [0 0 0]
  [0 0 0]]], [[[255 255 255]
  [255 255 255]
  [255 255 255]
  ...
  [255 255 255]
  [255 255 255]
  [255 255 255]]

 [[255 255 255]
  [255 255 255]
  [255 255 255]
  ...
  [255 255 255]
  [255 255 255]
  [255 255 255]]

 [[255 255 255]
  [255 255 255]
  [255 255 255]
  ...
  [255 255 255]
  [255 255 255]
  [255 255 255]]

 ...

 [[255 255 255]
  [255 255 255]
  [255 255 255]
  ...
  [255 255 255]
  [255 255 255]
  [255 255 255]]

 [[255 255 255]
  [255 255 255]
  [255 255 255]
  ...
  [255 255 255]
  [255 255 255]
  [255 255 255]]

 [[255 255 255]
  [255 255 255]
  [255 255 255]
  ...
 

# 2. Try to play 50 episodes before training (use the result as the base line of the model)

In [7]:
%%time
episodes = 50

for episode in range(1, episodes + 1):                     # looping from 1 to 50
    
    obs = env.reset()                                      # initial the set of observation
    
    done = False                                           # initial the game is over = False, until reach maximum number of steps in this particular environment
    
    score = 0                                              # running score counter
    
    while not done:
        
        env.render()                                       # view the graphical representation that environment
        
        action = env.action_space.sample()                 # random choose an action 
        
        n_obs, reward, done, info = env.step(action)       # pass random actions into environment to get back
                                                            # 1. next set of observation (4 observation in this case)
                                                            # 2. reward (Positive value increment, negative value decrement)
                                                            # 3. done (episode is done = True)
        
        score += reward                                    # accumulate each episodes' reward received into score
        
    print('Episode:{} Score:{}'.format(episode, score))    # print out score for each episode

2022-03-12 21:07:51.553 python[40856:797473] Warning: Expected min height of view: (<NSPopoverTouchBarItemButton: 0x7fbaaf9adf00>) to be less than or equal to 30 but got a height of 32.000000. This error will be logged once per view in violation.
2022-03-12 21:07:51.554 python[40856:797473] Warning: Expected min height of view: (<NSButton: 0x7fbab8b25210>) to be less than or equal to 30 but got a height of 32.000000. This error will be logged once per view in violation.
2022-03-12 21:07:51.556 python[40856:797473] Warning: Expected min height of view: (<NSPopoverTouchBarItemButton: 0x7fbab8b28bb0>) to be less than or equal to 30 but got a height of 32.000000. This error will be logged once per view in violation.
2022-03-12 21:07:51.558 python[40856:797473] Warning: Expected min height of view: (<NSPopoverTouchBarItemButton: 0x7fbab8b31dd0>) to be less than or equal to 30 but got a height of 32.000000. This error will be logged once per view in violation.


Episode:1 Score:160.0
Episode:2 Score:220.0
Episode:3 Score:210.0
Episode:4 Score:360.0
Episode:5 Score:180.0
Episode:6 Score:220.0
Episode:7 Score:270.0
Episode:8 Score:260.0
Episode:9 Score:150.0
Episode:10 Score:280.0
Episode:11 Score:250.0
Episode:12 Score:220.0
Episode:13 Score:130.0
Episode:14 Score:200.0
Episode:15 Score:240.0
Episode:16 Score:220.0
Episode:17 Score:140.0
Episode:18 Score:120.0
Episode:19 Score:310.0
Episode:20 Score:150.0
Episode:21 Score:270.0
Episode:22 Score:300.0
Episode:23 Score:170.0
Episode:24 Score:150.0
Episode:25 Score:190.0
Episode:26 Score:260.0
Episode:27 Score:280.0
Episode:28 Score:230.0
Episode:29 Score:160.0
Episode:30 Score:200.0
Episode:31 Score:160.0
Episode:32 Score:160.0
Episode:33 Score:300.0
Episode:34 Score:230.0
Episode:35 Score:300.0
Episode:36 Score:190.0
Episode:37 Score:210.0
Episode:38 Score:340.0
Episode:39 Score:210.0
Episode:40 Score:180.0
Episode:41 Score:110.0
Episode:42 Score:280.0
Episode:43 Score:100.0
Episode:44 Score:570

During the above 50 games, the score between 120-570 before training.

In [8]:
env.close()                             # close the opened enivernment

# 3. Vectorise Environment and Train Model

- Vectorizing the environment, particularly with multiple environments, allows us to train the agent faster by training in parallel

***Helper Functions***
- **make_atari_env** is a helper from stable baselines that helps create wrapped Atrai environments
- **VecFrameStack** allows us to stack the environemnts together

Policies
- Think of an agent's policy as the rule which tells it how to operate in the environment

Stable Baseline 3 has types of policy:
- MlpPolicy: Multi Layer Perceptrons Policy that implements actor critic, using a MLP (2 layers of 64)
- CnnPolicy: Convolution Neural Network Policy that implements actor critic, using a CNN (the nature CNN)

In [9]:
# train 32 environment at the same time

env = make_atari_env("MsPacman-v0", n_envs = 32, seed = 0)               # use make_atari_env to create Atari environment
                                                                          # n_envs: how many environment train at the same time

env = VecFrameStack(env, n_stack = 4)                                    # use VecFrameStack to set the number of frame stack together

In [10]:
log_path = os.path.join('Training', 'Logs')                               # set the log_path into ../Training/Logs

model = A2C('CnnPolicy', env, verbose = 1, tensorboard_log = log_path)    # using A2C model, save tensorboard_log in log_path
                                                                          # type of policy: CnnPolicy (for image training)

Using cpu device
Wrapping the env in a VecTransposeImage.


In [11]:
%%time
model.learn(total_timesteps = 1000000)                                    # Train 1M timesteps

Logging to Training/Logs/A2C_3
------------------------------------
| rollout/              |          |
|    ep_len_mean        | 908      |
|    ep_rew_mean        | 691      |
| time/                 |          |
|    fps                | 300      |
|    iterations         | 100      |
|    time_elapsed       | 53       |
|    total_timesteps    | 16000    |
| train/                |          |
|    entropy_loss       | -1.74    |
|    explained_variance | 0.116    |
|    learning_rate      | 0.0007   |
|    n_updates          | 99       |
|    policy_loss        | -0.195   |
|    value_loss         | 4.02     |
------------------------------------
------------------------------------
| rollout/              |          |
|    ep_len_mean        | 879      |
|    ep_rew_mean        | 679      |
| time/                 |          |
|    fps                | 314      |
|    iterations         | 200      |
|    time_elapsed       | 101      |
|    total_timesteps    | 32000    |
| train

------------------------------------
| rollout/              |          |
|    ep_len_mean        | 895      |
|    ep_rew_mean        | 769      |
| time/                 |          |
|    fps                | 325      |
|    iterations         | 1400     |
|    time_elapsed       | 688      |
|    total_timesteps    | 224000   |
| train/                |          |
|    entropy_loss       | -1.31    |
|    explained_variance | 0.857    |
|    learning_rate      | 0.0007   |
|    n_updates          | 1399     |
|    policy_loss        | -0.635   |
|    value_loss         | 4.09     |
------------------------------------
------------------------------------
| rollout/              |          |
|    ep_len_mean        | 909      |
|    ep_rew_mean        | 820      |
| time/                 |          |
|    fps                | 325      |
|    iterations         | 1500     |
|    time_elapsed       | 737      |
|    total_timesteps    | 240000   |
| train/                |          |
|

------------------------------------
| rollout/              |          |
|    ep_len_mean        | 847      |
|    ep_rew_mean        | 714      |
| time/                 |          |
|    fps                | 315      |
|    iterations         | 2800     |
|    time_elapsed       | 1417     |
|    total_timesteps    | 448000   |
| train/                |          |
|    entropy_loss       | -1.21    |
|    explained_variance | 0.875    |
|    learning_rate      | 0.0007   |
|    n_updates          | 2799     |
|    policy_loss        | 0.537    |
|    value_loss         | 4.25     |
------------------------------------
------------------------------------
| rollout/              |          |
|    ep_len_mean        | 853      |
|    ep_rew_mean        | 699      |
| time/                 |          |
|    fps                | 315      |
|    iterations         | 2900     |
|    time_elapsed       | 1470     |
|    total_timesteps    | 464000   |
| train/                |          |
|

------------------------------------
| rollout/              |          |
|    ep_len_mean        | 894      |
|    ep_rew_mean        | 837      |
| time/                 |          |
|    fps                | 318      |
|    iterations         | 4200     |
|    time_elapsed       | 2113     |
|    total_timesteps    | 672000   |
| train/                |          |
|    entropy_loss       | -0.516   |
|    explained_variance | 0.949    |
|    learning_rate      | 0.0007   |
|    n_updates          | 4199     |
|    policy_loss        | -0.416   |
|    value_loss         | 5.87     |
------------------------------------
------------------------------------
| rollout/              |          |
|    ep_len_mean        | 904      |
|    ep_rew_mean        | 854      |
| time/                 |          |
|    fps                | 318      |
|    iterations         | 4300     |
|    time_elapsed       | 2162     |
|    total_timesteps    | 688000   |
| train/                |          |
|

------------------------------------
| rollout/              |          |
|    ep_len_mean        | 1.02e+03 |
|    ep_rew_mean        | 1.07e+03 |
| time/                 |          |
|    fps                | 319      |
|    iterations         | 5600     |
|    time_elapsed       | 2800     |
|    total_timesteps    | 896000   |
| train/                |          |
|    entropy_loss       | -0.555   |
|    explained_variance | 0.972    |
|    learning_rate      | 0.0007   |
|    n_updates          | 5599     |
|    policy_loss        | -0.118   |
|    value_loss         | 2.77     |
------------------------------------
------------------------------------
| rollout/              |          |
|    ep_len_mean        | 1.04e+03 |
|    ep_rew_mean        | 1.12e+03 |
| time/                 |          |
|    fps                | 320      |
|    iterations         | 5700     |
|    time_elapsed       | 2849     |
|    total_timesteps    | 912000   |
| train/                |          |
|

Spend 58min 36s to train 1M timesteps as the above

Notice that: the ***explained_variance 0.97*** (very close to 1) ~ good sign!

# 4. Save the Model

In [12]:
a2c_path = os.path.join('Training', 'Saved Models', 'A2C_MsPacman_Model_1Mtimesteps')  # set a2c_path into ../Training/Saved Models/A2C_MsPacman_Model_1Mtimesteps

model.save(a2c_path)                                                                   # save the trained model

/Users/hoikinyu/opt/anaconda3/lib/python3.8/site-packages/stable_baselines3/common/save_util.py:276: UserWarning: Path 'Training/Saved Models' does not exist. Will create it.
  warnings.warn(f"Path '{path.parent}' does not exist. Will create it.")


# 5. Evaluate the Model

In [14]:
env = make_atari_env("MsPacman-v0", n_envs = 1, seed = 0)     # reset the n_envs = 1 for evaluate and test purpose

env = VecFrameStack(env, n_stack = 4)                         # n_stack: number of frame stack together

In [15]:
evaluate_policy(model, env, n_eval_episodes = 10, render = True)  # use evaluate_policy to evaluate the trained model
                                                                   # n_eval_episodes: number of evaluate episodes

2022-03-12 22:34:26.468 python[40856:797473] Warning: Expected min height of view: (<NSPopoverTouchBarItemButton: 0x7fba88586b60>) to be less than or equal to 30 but got a height of 32.000000. This error will be logged once per view in violation.
2022-03-12 22:34:26.469 python[40856:797473] Warning: Expected min height of view: (<NSButton: 0x7fba7ffeffa0>) to be less than or equal to 30 but got a height of 32.000000. This error will be logged once per view in violation.
2022-03-12 22:34:26.472 python[40856:797473] Warning: Expected min height of view: (<NSPopoverTouchBarItemButton: 0x7fba7fff27b0>) to be less than or equal to 30 but got a height of 32.000000. This error will be logged once per view in violation.
2022-03-12 22:34:26.474 python[40856:797473] Warning: Expected min height of view: (<NSPopoverTouchBarItemButton: 0x7fba7fff96d0>) to be less than or equal to 30 but got a height of 32.000000. This error will be logged once per view in violation.


(1014.0, 202.59318843435975)

# 6. Viewing Logs in Tensorboard

### Two Core Evaluation Metrics we should pay attention to: 

***ep_len_mean:*** on average how long a particular episode lasted before gameover

***ep_rew_mean:*** the average reward that the agent accumulated per episode

In [16]:
training_log_path = os.path.join(log_path, 'A2C_3')

!tensorboard --logdir={training_log_path}


NOTE: Using experimental fast data loading logic. To disable, pass
    "--load_fast=false" and report issues on GitHub. More details:
    https://github.com/tensorflow/tensorboard/issues/4784

Serving TensorBoard on localhost; to expose to the network, use a proxy or pass --bind_all
TensorBoard 2.6.0 at http://localhost:6006/ (Press CTRL+C to quit)
^C


![A2C_MsPacman_Model_1Mtimesteps?](images/A2C_MsPacman_Model_1Mtimesteps.png)

The result on the above show that:
- Starting from step 700k, the agent performance improved significantly
- The average lasted from 900 episode in 700k step, rised to more than 1.1e+3 episode in 1M step
- The average reward from 850 score in 700k step, rised to more that 1.1e+3 score in 1M step